Shravan P
1RVU22BSC095

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 1️⃣ Build a Simple CNN Model
def create_model():
    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), name="conv1"),
        layers.MaxPooling2D((2, 2), name="pool1"),
        layers.Conv2D(64, (3, 3), activation='relu', name="conv2"),
        layers.MaxPooling2D((2, 2), name="pool2"),
        layers.Flatten(name="flatten"),
        layers.Dense(128, activation='relu', name="dense1"),
        layers.Dense(10, activation='softmax', name="output")
    ])
    return model

# 2️⃣ Load Dataset (MNIST)
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# 3️⃣ Train the Baseline Model
model = create_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("\n🔹 Baseline Model Summary:")
model.summary()

print("\n🚀 Training the baseline model...")
model.fit(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

# 4️⃣ Evaluate the Baseline Model
baseline_accuracy = model.evaluate(x_test, y_test, verbose=0)[1]
print(f"✅ Baseline Model Accuracy: {baseline_accuracy:.4f}")

# 5️⃣ Define Filter Pruning Function
def prune_conv_layer(conv_layer, prune_ratio=0.5):
    """Prunes filters with the lowest L1 norm from a Conv2D layer."""
    weights = conv_layer.get_weights()
    kernel, bias = weights[0], weights[1]

    num_filters = kernel.shape[-1]  # Number of filters

    # Compute L1 norm of each filter
    l1_norms = np.sum(np.abs(kernel), axis=(0, 1, 2))

    # Get indices of filters to KEEP
    num_prune = int(num_filters * prune_ratio)
    keep_indices = np.argsort(l1_norms)[num_prune:]

    # Keep only selected filters
    new_kernel = kernel[..., keep_indices]
    new_bias = bias[keep_indices]

    # Create a new Conv2D layer with fewer filters
    new_conv_layer = tf.keras.layers.Conv2D(
        filters=len(keep_indices),
        kernel_size=conv_layer.kernel_size,
        strides=conv_layer.strides,
        padding=conv_layer.padding,
        activation=conv_layer.activation,
        use_bias=(bias is not None),
        name=conv_layer.name + "_pruned"
    )

    # Apply new weights to the layer
    new_conv_layer.build(conv_layer.input.shape)
    new_conv_layer.set_weights([new_kernel, new_bias])

    return new_conv_layer, len(keep_indices)

# 6️⃣ Apply Pruning to the First Convolutional Layer
pruned_conv1, new_filters = prune_conv_layer(model.layers[0], prune_ratio=0.8)

# 7️⃣ Modify Pooling Layer and Next Convolutional Layer
pruned_pool1 = layers.MaxPooling2D((2, 2), name="pool1_pruned")

# **FIX: Explicitly define input shape for conv2_pruned**
pruned_conv2 = layers.Conv2D(
    filters=64, kernel_size=(3, 3), activation='relu',
    input_shape=(13, 13, new_filters),  # **Ensure correct input shape**
    name="conv2_pruned"
)

pruned_pool2 = layers.MaxPooling2D((2, 2), name="pool2_pruned")

# **FIX: Ensure Flatten layer adapts to new shape**
pruned_flatten = layers.Flatten(name="flatten_pruned")

# 8️⃣ Rebuild Model with Pruned Layers
new_model = keras.Sequential([
    layers.Input(shape=(28, 28, 1)),  # **Ensure input shape is set**
    pruned_conv1,
    pruned_pool1,
    pruned_conv2,
    pruned_pool2,
    pruned_flatten,
    model.layers[5],  # Dense
    model.layers[6],  # Output Layer
])

# 9️⃣ Show Pruned Model Summary
print("\n🔹 Pruned Model Summary:")
new_model.build(input_shape=(None, 28, 28, 1))  # **Ensure layers are built**
new_model.summary()

# 🔟 Compile and Retrain the Pruned Model
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("\n🚀 Retraining the pruned model...")
new_model.fit(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

# 🔟 Evaluate the Pruned Model
pruned_accuracy = new_model.evaluate(x_test, y_test, verbose=0)[1]
print(f"✅ Pruned Model Accuracy: {pruned_accuracy:.4f}")

# 🔍 Compare Results
print(f"\n📊 Accuracy Before Pruning: {baseline_accuracy:.4f}")
print(f"📊 Accuracy After Pruning: {pruned_accuracy:.4f}")

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

🔹 Baseline Model Summary:


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1 (Conv2D)                  │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool1 (MaxPooling2D)            │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv2D)                  │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool2 (MaxPooling2D)            │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense1 (Dense)                  │ (None, 128)            │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 225,034 (879.04 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1_pruned (Conv2D)           │ (None, 26, 26, 7)      │            70 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool1_pruned (MaxPooling2D)     │ (None, 13, 13, 7)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2_pruned (Conv2D)           │ (None, 11, 11, 64)     │         4,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool2_pruned (MaxPooling2D)     │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_pruned (Flatten)        │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense1 (Dense)                  │ (None, 128)            │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 210,384 (821.81 KB)

 Trainable params: 210,384 (821.81 KB)

 Non-trainable params: 0 (0.00 B)


🚀 Retraining the pruned model...
Epoch 1/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 38s 19ms/step - accuracy: 0.9466 - loss: 0.1888 - val_accuracy: 0.9872 - val_loss: 0.0383
Epoch 2/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 36s 19ms/step - accuracy: 0.9885 - loss: 0.0357 - val_accuracy: 0.9898 - val_loss: 0.0326
✅ Pruned Model Accuracy: 0.9898

📊 Accuracy Before Pruning: 0.9895
📊 Accuracy After Pruning: 0.9898


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 1️⃣ Build a Simple CNN Model
def create_model():
    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), name="conv1"),
        layers.MaxPooling2D((2, 2), name="pool1"),
        layers.Conv2D(64, (3, 3), activation='relu', name="conv2"),
        layers.MaxPooling2D((2, 2), name="pool2"),
        layers.Flatten(name="flatten"),
        layers.Dense(128, activation='relu', name="dense1"),
        layers.Dense(10, activation='softmax', name="output")
    ])
    return model

# 2️⃣ Load Dataset (MNIST)
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# 3️⃣ Train the Baseline Model
model = create_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("\n🔹 Baseline Model Summary:")
model.summary()

print("\n🚀 Training the baseline model...")
model.fit(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

# 4️⃣ Evaluate the Baseline Model
baseline_accuracy = model.evaluate(x_test, y_test, verbose=0)[1]
print(f"✅ Baseline Model Accuracy: {baseline_accuracy:.4f}")

# 5️⃣ Define Filter Pruning Function
def prune_conv_layer(conv_layer, prune_ratio=0.5):
    """Prunes filters with the lowest L1 norm from a Conv2D layer."""
    weights = conv_layer.get_weights()
    kernel, bias = weights[0], weights[1]

    num_filters = kernel.shape[-1]  # Number of filters

    # Compute L1 norm of each filter
    l1_norms = np.sum(np.abs(kernel), axis=(0, 1, 2))

    # Get indices of filters to KEEP
    num_prune = int(num_filters * prune_ratio)
    keep_indices = np.argsort(l1_norms)[num_prune:]

    # Keep only selected filters
    new_kernel = kernel[..., keep_indices]
    new_bias = bias[keep_indices]

    # Create a new Conv2D layer with fewer filters
    new_conv_layer = layers.Conv2D(
        filters=len(keep_indices),
        kernel_size=conv_layer.kernel_size,
        strides=conv_layer.strides,
        padding=conv_layer.padding,
        activation=conv_layer.activation,
        use_bias=True,
        name=conv_layer.name + "_pruned"
    )

    # Apply new weights to the layer
    input_depth = kernel.shape[2]  # Preserve input depth
    new_conv_layer.build((None, None, None, input_depth))
    new_conv_layer.set_weights([new_kernel, new_bias])

    return new_conv_layer, len(keep_indices)

# 6️⃣ Apply Pruning to the First Convolutional Layer
pruned_conv1, filters_conv1 = prune_conv_layer(model.layers[0], prune_ratio=0.8)

# 7️⃣ Modify Pooling Layer to Match the New Output
pruned_pool1 = layers.MaxPooling2D((2, 2), name="pool1_pruned")

# 8️⃣ Prune the Second Convolutional Layer to Match `conv1_pruned` Output Depth
original_conv2 = model.layers[2]  # Get the second Conv2D layer
pruned_conv2, filters_conv2 = prune_conv_layer(original_conv2, prune_ratio=0.5)

# 🔍 Fix Input Depth Mismatch
pruned_conv2 = layers.Conv2D(
    filters=filters_conv2,
    kernel_size=original_conv2.kernel_size,
    strides=original_conv2.strides,
    padding=original_conv2.padding,
    activation=original_conv2.activation,
    use_bias=True,
    name=original_conv2.name + "_pruned"
)

# Ensure the input shape matches the output of `pruned_conv1`
pruned_conv2.build((None, None, None, pruned_conv1.filters))

# 9️⃣ Modify Pooling Layer to Match the New Output
pruned_pool2 = layers.MaxPooling2D((2, 2), name="pool2_pruned")

# 🔟 Compute the Flattened Output Shape Dynamically
dummy_input = np.random.randn(1, 28, 28, 1).astype(np.float32)  # Sample input batch
intermediate_model = keras.Sequential([
    layers.Input(shape=(28, 28, 1)),
    pruned_conv1,
    pruned_pool1,
    pruned_conv2,
    pruned_pool2,
    layers.Flatten()
])

# Get the output shape of the Flatten layer dynamically
dummy_output = intermediate_model.predict(dummy_input)
flattened_dim = dummy_output.shape[1]  # Correct dimension for Dense layer

# 🔟 Rebuild Model with Pruned Layers
new_model = keras.Sequential([
    layers.Input(shape=(28, 28, 1)),
    pruned_conv1,
    pruned_pool1,
    pruned_conv2,
    pruned_pool2,
    layers.Flatten(),
    layers.Dense(128, activation='relu', name="dense1_pruned"),  # Fixing input shape
    layers.Dense(10, activation='softmax', name="output")  # Output Layer
])

# 🔟 Show Pruned Model Summary
print("\n🔹 Pruned Model Summary:")
new_model.build(input_shape=(None, 28, 28, 1))
new_model.summary()

# 🔟 Compile and Retrain the Pruned Model
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("\n🚀 Retraining the pruned model...")
new_model.fit(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

# 🔟 Evaluate the Pruned Model
pruned_accuracy = new_model.evaluate(x_test, y_test, verbose=0)[1]
print(f"✅ Pruned Model Accuracy: {pruned_accuracy:.4f}")

# 🔍 Compare Results
print(f"\n📊 Accuracy Before Pruning: {baseline_accuracy:.4f}")
print(f"📊 Accuracy After Pruning: {pruned_accuracy:.4f}")


🔹 Baseline Model Summary:


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1 (Conv2D)                  │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool1 (MaxPooling2D)            │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv2D)                  │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool2 (MaxPooling2D)            │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense1 (Dense)                  │ (None, 128)            │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 225,034 (879.04 KB)

 Trainable params: 225,034 (879.04 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1_pruned (Conv2D)           │ (None, 26, 26, 7)      │            70 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool1_pruned (MaxPooling2D)     │ (None, 13, 13, 7)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2_pruned (Conv2D)           │ (None, 11, 11, 32)     │         2,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool2_pruned (MaxPooling2D)     │ (None, 5, 5, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 800)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense1_pruned (Dense)           │ (None, 128)            │       102,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 105,936 (413.81 KB)

 Trainable params: 105,936 (413.81 KB)

 Non-trainable params: 0 (0.00 B)


🚀 Retraining the pruned model...
Epoch 1/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 32s 16ms/step - accuracy: 0.9085 - loss: 0.3201 - val_accuracy: 0.9829 - val_loss: 0.0555
Epoch 2/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 43s 18ms/step - accuracy: 0.9845 - loss: 0.0497 - val_accuracy: 0.9866 - val_loss: 0.0383
✅ Pruned Model Accuracy: 0.9866

📊 Accuracy Before Pruning: 0.9853
📊 Accuracy After Pruning: 0.9866


In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# Define the model
def create_model():
    inputs = keras.Input(shape=(28, 28, 1))
    x = layers.Conv2D(32, (3, 3), activation='relu', name="conv1")(inputs)
    x = layers.MaxPooling2D((2, 2), name="pool1")(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', name="conv2")(x)
    x = layers.MaxPooling2D((2, 2), name="pool2")(x)
    x = layers.Flatten(name="flatten")(x)
    x = layers.Dense(128, activation='relu', name="dense1")(x)
    outputs = layers.Dense(10, activation='softmax', name="output")(x)

    model = keras.Model(inputs, outputs)
    return model

# Create and compile the model
model = create_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

# Function to prune convolutional filters based on L1 norm
def l1_norm_prune_conv_layer(layer, prune_ratio=0.5):
    weights, biases = layer.get_weights()
    l1_norms = np.sum(np.abs(weights), axis=(0, 1, 2))
    num_filters_to_keep = int(weights.shape[-1] * (1 - prune_ratio))

    # Get indices of filters to keep
    top_indices = np.argsort(l1_norms)[-num_filters_to_keep:]

    # Prune filters
    pruned_weights = weights[:, :, :, top_indices]
    pruned_biases = biases[top_indices]

    # Create new layer with pruned filters
    pruned_layer = layers.Conv2D(
        num_filters_to_keep,
        layer.kernel_size,
        activation=layer.activation,
        padding=layer.padding,
        strides=layer.strides,
        name=layer.name + "_pruned"
    )

    return pruned_layer, pruned_weights, pruned_biases

# Apply pruning to the first convolutional layer
pruned_conv1, pruned_weights, pruned_biases = l1_norm_prune_conv_layer(model.get_layer("conv1"), prune_ratio=0.5)
print(f"Pruned Conv1 Layer: {pruned_conv1.name}, Remaining Filters: {pruned_weights.shape[-1]}")

# Rebuild the model with pruned layers
def rebuild_pruned_model():
    inputs = keras.Input(shape=(28, 28, 1))
    x = pruned_conv1(inputs)
    x = layers.MaxPooling2D((2, 2), name="pool1")(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', name="conv2")(x)
    x = layers.MaxPooling2D((2, 2), name="pool2")(x)
    x = layers.Flatten(name="flatten")(x)
    x = layers.Dense(128, activation='relu', name="dense1")(x)
    outputs = layers.Dense(10, activation='softmax', name="output")(x)

    pruned_model = keras.Model(inputs, outputs)
    return pruned_model

# Build and compile pruned model
pruned_model = rebuild_pruned_model()
pruned_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Assign the pruned weights
pruned_model.get_layer(pruned_conv1.name).set_weights([pruned_weights, pruned_biases])

# Train the pruned model
pruned_model.fit(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

# Evaluate performance
baseline_accuracy = model.evaluate(x_test, y_test, verbose=0)[1]
pruned_accuracy = pruned_model.evaluate(x_test, y_test, verbose=0)[1]

print(f" Baseline Model Accuracy: {baseline_accuracy:.4f}")
print(f" Pruned Model Accuracy: {pruned_accuracy:.4f}")

Epoch 1/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 57s 30ms/step - accuracy: 0.9035 - loss: 0.3004 - val_accuracy: 0.9845 - val_loss: 0.0515
Epoch 2/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 56s 30ms/step - accuracy: 0.9848 - loss: 0.0473 - val_accuracy: 0.9907 - val_loss: 0.0312
Pruned Conv1 Layer: conv1_pruned, Remaining Filters: 16
Epoch 1/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 43s 22ms/step - accuracy: 0.9225 - loss: 0.2625 - val_accuracy: 0.9877 - val_loss: 0.0377
Epoch 2/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 81s 22ms/step - accuracy: 0.9884 - loss: 0.0372 - val_accuracy: 0.9906 - val_loss: 0.0311
 Baseline Model Accuracy: 0.9907
 Pruned Model Accuracy: 0.9906
